## EMOTION DETECTION FROM TEXT USING NLTK

IMPORT PACKAGES

In [4]:
import nltk
nltk.download('stopwords')

from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


from transformers import pipeline

import pandas as pd
df= pd.DataFrame(data)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


TEXT EMOTION

In [5]:
data = [
    {"text": "I am feeling really great today!", "emotion": "happy"},
    {"text": "This is the best day ever!", "emotion": "happy"},
    {"text": "I am so excited and full of joy.", "emotion": "happy"},
    {"text": "Your message made my whole day.", "emotion": "happy"},

    {"text": "I feel really down today.", "emotion": "sad"},
    {"text": "Nothing seems to make me feel better.", "emotion": "sad"},
    {"text": "I miss those days, it hurts.", "emotion": "sad"},
    {"text": "I feel lonely and empty inside.", "emotion": "sad"},

    {"text": "I am so frustrated right now.", "emotion": "angry"},
    {"text": "Why does this always happen? It makes me furious!", "emotion": "angry"},
    {"text": "I can’t tolerate this nonsense anymore.", "emotion": "angry"},
    {"text": "This situation really pisses me off.", "emotion": "angry"},

    {"text": "I’m scared of what might happen next.", "emotion": "fear"},
    {"text": "My heart is racing, I feel terrified.", "emotion": "fear"},
    {"text": "I’m really worried and anxious about this.", "emotion": "fear"},
    {"text": "Something feels wrong, I’m afraid.", "emotion": "fear"},

    {"text": "That was the most disgusting thing I’ve ever seen.", "emotion": "disgust"},
    {"text": "I feel sick just thinking about it.", "emotion": "disgust"},
    {"text": "This makes me extremely uncomfortable.", "emotion": "disgust"},
    {"text": "I can’t stand how gross this is.", "emotion": "disgust"},

    {"text": "Wow, I didn’t expect that at all!", "emotion": "surprise"},
    {"text": "That was completely unexpected.", "emotion": "surprise"},
    {"text": "I’m shocked this happened.", "emotion": "surprise"},
    {"text": "This took me by surprise, honestly.", "emotion": "surprise"}
]


TEXT PROCESSING AND CLEANING


In [6]:
def clean_text(text):
  text= text.lower()

  return text

df['text']=df['text'].apply(clean_text)


TRAIN EMOTION DATASET USING NAIVE BAYES CLASSIFIER


In [7]:

# X = text, y = label
X = df['text']
y = df['emotion']          # ✅ column name is 'emotion', not 'emotions'

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42  # ✅ correct arg name
)

# TF-IDF vectorizer
vectorizer = TfidfVectorizer()            # ✅ need ()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Naive Bayes classifier
classifier = MultinomialNB()
classifier.fit(X_train_vec, y_train)

# optional: check accuracy
y_pred = classifier.predict(X_test_vec)
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.0


ACCURACY SCORE

In [8]:
# make predictions
pred = classifier.predict(X_test_vec)

print(pred)


['happy' 'surprise' 'sad' 'surprise' 'sad']


HUGGING FACE EMOTION DETECTION


In [9]:
from transformers import pipeline

# load a pretrained emotion classifier
# (works well for English emotions)
emotion_model = pipeline(
    "text-classification",
    model="bhadresh-savani/distilbert-base-uncased-emotion"  # or another emotion model
)

def hf_predict_emotion(sentence):
    result = emotion_model(sentence)[0]   # [{'label': 'anger', 'score': 0.98}, ...]
    return result['label'], result['score']

test_sentence = "I am so frustrated and angry right now."
label, score = hf_predict_emotion(test_sentence)

print("Text:", test_sentence)
print("Predicted emotion:", label)
print("Confidence:", score)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


Text: I am so frustrated and angry right now.
Predicted emotion: anger
Confidence: 0.9979800581932068


In [10]:
print(hf_predict_emotion("my babeee is so happy today "))

('joy', 0.9985827207565308)


In [11]:
print(hf_predict_emotion("but she was angery yesterday "))

('anger', 0.9971522092819214)
